# Why is Nipype so cool?

Let's consider a very simple fMRI preprocessing workflow that performs:
 1. slice time correction
 2. motion correction
 3. smoothing

# Preparing the preprocessing workflow

In [ ]:
# Import Nodes and Workflows
from nipype import Node, Workflow

In [ ]:
# Import interfaces to perform preprocessing
from nipype.interfaces.fsl import SliceTimer, MCFLIRT, Smooth

In [ ]:
# Initiate a node to correct for slice wise acquisition
slicetimer = Node(SliceTimer(index_dir=False,
                             interleaved=True,
                             time_repetition=2.5),
                  name="slicetimer")

In [ ]:
# Initiate a node to correct for motion
mcflirt = Node(MCFLIRT(mean_vol=True,
                       save_plots=True),
               name="mcflirt")

In [ ]:
# Initiate a node to smooth functional images
smooth = Node(Smooth(fwhm=4), name="smooth")

In [ ]:
# Create workflow - a place where to put the preprocessing nodes
preproc01 = Workflow(name='preproc01', base_dir='.')

In [ ]:
# Connect the preprocessing nodes in a workflow
preproc01.connect([(slicetimer, mcflirt, [('slice_time_corrected_file', 'in_file')]),
                   (mcflirt, smooth, [('out_file', 'in_file')])])

In [ ]:
# Write the graph of the workflow to a file
preproc01.write_graph(graph2use='flat')

In [ ]:
# Visualize graph
from IPython.display import Image
Image(filename="preproc01/graph_detailed.dot.png")

# Run the workflow on one functional image

Now, that we create our workflow, let's run it on a functional image. For this we first need to specify the input file of the very first node, i.e. the `slicetimer` node.

In [ ]:
slicetimer.inputs.in_file = '/data/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-fingerfootlips_bold.nii.gz'

To show off Nipype's parallelization power, let's run the workflow in parallel, on 5 processors and let's stop the execution time:

In [ ]:
%time preproc01.run('MultiProc', plugin_args={'n_procs': 5})

Nice, the whole execution took ~2min. But wait... The parallelization didn't really help.

That's true, but because there was no possibility to run the workflow in parallel. Each node depends on the output of the previous node.

# Results of `preproc01`

So, what did we get? Let's look at the output folder `preproc01`:

In [ ]:
!tree preproc01 -I '*js|*json|*pklz|_report|*.dot|*html'

# Rerunning of a workflow

Now, for fun. Let's run the workflow again, but let's change the `fwhm` value of the Gaussian smoothing kernel to `2`.

In [ ]:
smooth.inputs.fwhm = 2

And let's run the workflow again.

In [ ]:
%time preproc01.run('MultiProc', plugin_args={'n_procs': 5})

Interesting, now it only took ~15s to execute the whole workflow again. **What happened?**

As you can see from the log above is that Nipype didn't execute the two nodes `slicetimer` and `mclfirt`, because their input values didn't change from the last execution. The `preproc01` workflow therefore only had to rerun the node `smooth`.

# Running a workflow in parallel

Ok, ok... Rerunning a workflow again is faster. That's nice and all, but I want more. You spoke of parallel execution.

We saw that the `preproc01` workflow takes about ~2min to execute completely. So, if we would run the workflow on five functional images, it should take about ~10min. Assuming that the execution is done sequentially. Let's see how long it takes if we run it in parallel.

In [ ]:
# First, let's copy/clone 'preproc01'
preproc02 = preproc01.clone('preproc02')
preproc03 = preproc01.clone('preproc03')
preproc04 = preproc01.clone('preproc04')
preproc05 = preproc01.clone('preproc05')

In [ ]:
# Now, let's create a new workflow that will contain the five preproc workflows
metaflow = Workflow(name='metaflow', base_dir='.')

In [ ]:
# Now we can add the five preproc workflows to the bigger metaflow
metaflow.add_nodes([preproc01, preproc02, preproc03,
                    preproc04, preproc05])

In [ ]:
# As before, let's write the graph of the workflow
metaflow.write_graph(graph2use='flat')

In [ ]:
# And visualize the graph
from IPython.display import Image
Image(filename="metaflow/graph_detailed.dot.png")

Ah... so now we can see that the `metaflow` has potential for parallelization. So let's put it to test

In [ ]:
%time metaflow.run('MultiProc', plugin_args={'n_procs': 5})

This time we can see that Nipype uses all available processors.

And if all went well, the total execution time should still be around ~2min. That's why Nipype is so amazing. No more the need of opening multiple SPMs, FSLs, AFNIs etc.

# Results of `metaflow`

In [ ]:
!tree metaflow -I '*js|*json|*pklz|_report|*.dot|*html'